In [ ]:
import requests, json, pprint
import pandas as pd

def get_address(lat, lng):
    url = "https://dapi.kakao.com/v2/local/geo/coord2regioncode.json?x="+lng+"&y="+lat
    # 'KaKaoAK '는 그대로 두고 개인키만 지우고 입력
    headers = {"Authorization": "KakaoAK e1126f906645b3229186076b82390eee"}
    api_json = requests.get(url, headers=headers)
    full_address = json.loads(api_json.text)

    return full_address

In [ ]:
bus = pd.read_csv('/content/drive/MyDrive/응통 분석 공모전/데이터/주변시설 데이터/서울시버스정류소좌표데이터(2022.08.24).csv',encoding='cp949')
sub = pd.read_csv('/content/drive/MyDrive/응통 분석 공모전/데이터/주변시설 데이터/서울시 역사마스터 정보.csv',encoding='cp949')

In [ ]:
bus.head()

,NODE_ID,ARS-ID,정류소명,좌표X,좌표Y
0,100000001,1001,종로2가사거리,126.987786,37.569764
1,100000002,1002,창경궁.서울대학교병원,126.996520,37.579179
2,100000003,1003,명륜3가.성대입구,126.998290,37.582709
3,100000004,1004,종로2가.삼일교,126.987507,37.568582
4,100000005,1005,혜화동로터리.여운형활동터,127.001694,37.586230


In [ ]:
sub.head()

,역사_ID,역사명,호선,위도,경도
0,9996,미사,5호선,127.193877,37.560927
1,9995,강일,5호선,127.175930,37.557490
2,4929,김포공항,김포골드라인,126.801868,37.562360
3,4928,고촌,김포골드라인,126.770345,37.601243
4,4927,풍무,김포골드라인,126.732387,37.612488


In [ ]:
bus = bus.astype({'좌표X':'string'})
bus = bus.astype({'좌표Y':'string'})

sub = sub.astype({'위도':'string'})
sub = sub.astype({'경도':'string'})

In [ ]:
#순서대로 행정동코드, 동, 시, 구
bus["code"]=""
bus["region"]=""
bus['city']=""
bus['bigCity']=""

sub["code"]=""
sub["region"]=""
sub['city']=""
sub['bigCity']=""

In [ ]:
bus.head()

,NODE_ID,ARS-ID,정류소명,좌표X,좌표Y,code,region,city,bigCity
0,100000001,1001,종로2가사거리,126.9877861534,37.5697641508,,,,
1,100000002,1002,창경궁.서울대학교병원,126.9965201597,37.5791788583,,,,
2,100000003,1003,명륜3가.성대입구,126.9982901625,37.5827088189,,,,
3,100000004,1004,종로2가.삼일교,126.9875072018,37.5685822333,,,,
4,100000005,1005,혜화동로터리.여운형활동터,127.0016943494,37.5862299819,,,,


In [ ]:
sub.head()

,역사_ID,역사명,호선,위도,경도,code,region,city,bigCity
0,9996,미사,5호선,127.193877,37.560927,,,,
1,9995,강일,5호선,127.17593,37.55749,,,,
2,4929,김포공항,김포골드라인,126.801868,37.56236,,,,
3,4928,고촌,김포골드라인,126.770345,37.601243,,,,
4,4927,풍무,김포골드라인,126.732387,37.612488,,,,


## subway

In [ ]:
#카카오 api 예시
full_address = get_address(sub['경도'][749], sub['위도'][749])
address = full_address['documents']
pprint.pprint(address[-1]['code'])
pprint.pprint(address[-1]['region_3depth_name'])
pprint.pprint(address)

'1114059000'
'광희동'
[{'address_name': '서울특별시 중구 을지로7가',
  'code': '1114014900',
  'region_1depth_name': '서울특별시',
  'region_2depth_name': '중구',
  'region_3depth_name': '을지로7가',
  'region_4depth_name': '',
  'region_type': 'B',
  'x': 127.00974335634676,
  'y': 37.566149983715114},
 {'address_name': '서울특별시 중구 광희동',
  'code': '1114059000',
  'region_1depth_name': '서울특별시',
  'region_2depth_name': '중구',
  'region_3depth_name': '광희동',
  'region_4depth_name': '',
  'region_type': 'H',
  'x': 127.00505266779864,
  'y': 37.56446721686456}]


In [ ]:
for i in range(len(sub)):
  full_address = get_address(sub['경도'][i], sub['위도'][i])
  address = full_address['documents'][-1]
  sub['code'][i] = address['code']
  sub['region'][i] = address['region_3depth_name']
  sub['city'][i] = address['region_1depth_name']
  sub['bigCity'][i] = address['region_2depth_name']

In [ ]:
#각 동에 위치한 지하철역의 개수 세기
sub['region'].value_counts()

역삼1동           7
여의동            6
종로1.2.3.4가동    6
문정2동           5
용신동            5
              ..
방이1동           1
가락본동           1
목3동            1
거여1동           1
종로5.6가동        1
Name: region, Length: 228, dtype: int64

In [ ]:
#결과 내보내기
sub.to_csv('sub_dataT.csv',encoding='utf-8-sig')

## bus

In [ ]:
#카카오 api 예시
full_address = get_address(bus['좌표Y'][0], bus['좌표X'][0])
pprint.pprint(full_address['documents'][-1])

{'address_name': '서울특별시 종로구 종로1.2.3.4가동',
 'code': '1111061500',
 'region_1depth_name': '서울특별시',
 'region_2depth_name': '종로구',
 'region_3depth_name': '종로1.2.3.4가동',
 'region_4depth_name': '',
 'region_type': 'H',
 'x': 126.99028726084354,
 'y': 37.574435539666744}


In [ ]:
for i in range(len(bus)):
  full_address = get_address(bus['좌표Y'][i], bus['좌표X'][i])
  address = full_address['documents'][-1]
  bus['code'][i] = address['code']
  bus['region'][i] = address['region_3depth_name']
  bus['city'][i] = address['region_1depth_name']
  bus['bigCity'][i] = address['region_2depth_name']

In [ ]:
bus['region'].value_counts()

In [ ]:
bus.to_csv('bus_dataT.csv',encoding='utf-8-sig')

## 예외처리 (신사동, 상일동)<br>
신사동의 경우 강남구, 관악구 두개의 구에 위치한다 (카카오 api를 신사동일 경우에만 다시 불러오기)<br>
생활인구 데이터와 맞추어, 상일1동, 상일2동은 상일동으로 합친다 (그냥 덧셈)

In [ ]:
for i in range(len(bus)):
  full_address = get_address(bus['좌표Y'][i], bus['좌표X'][i])
  address = full_address['documents'][-1]
  if address['region_3depth_name']=='신사동':
    bus['code'][i] = address['code']
    bus['region'][i] = address['region_3depth_name']
    bus['city'][i] = address['region_1depth_name']
    bus['bigCity'][i] = address['region_2depth_name']

In [ ]:
for i in range(len(sub)):
  full_address = get_address(sub['경도'][i], sub['위도'][i])
  address = full_address['documents'][-1]
  if address['region_3depth_name']=='신사동':
    sub['code'][i] = address['code']
    sub['region'][i] = address['region_3depth_name']
    sub['city'][i] = address['region_1depth_name']
    sub['bigCity'][i] = address['region_2depth_name']